### Historical Product Demand - Analysis & Models
**Data Source:** [Kaggle](https://www.kaggle.com/felixzhao/productdemandforecasting)

**Data Description from Kaggle:** *The dataset contains historical product demand for a manufacturing company with footprints globally. The company provides thousands of products within dozens of product categories. There are four central warehouses to ship products within the region it is responsible for. Since the products are manufactured in different locations all over the world, it normally takes more than one month to ship products via ocean to different central warehouses. If forecasts for each product in different central with reasonable accuracy for the monthly demand for month after next can be achieved, it would be beneficial to the company in multiple ways.*

**Objective:** *Is it possible to make forecasts for thousands of products (some of them are highly variable in terms of monthly demand) for the the month after next?* <br/>
The latest data month is Jan 2017, thus forecast is for Mar 2017 (onwards). <br/>

RMSE is used as performance metric in this forecast.

**Assumptions:** <br/>
A1/ Date refers to shipping date, not order date. Otherwise, forecast cannot be done. Shipping date indicates the time when products need to be avaiable at these warehouse. <br/>
A2/ Order demand refers to the order quantity by customers (actual demands), not the order quantity that can be fulfilled by warehouses. <br/>
Other assumptions will be made throughout the analysis. 

In [1]:
# Load packagse
import pandas as pd
import numpy as np
from operator import attrgetter
#pip install pmdarima
from pmdarima.arima import auto_arima
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing

In [2]:
# Read data
data = pd.read_csv("Historical Product Demand.csv")
data.dtypes

Product_Code        object
Warehouse           object
Product_Category    object
Date                object
Order_Demand        object
dtype: object

1/ All columns are formatted as characters and need to be reformatted: <br/> 
-**Date** to datetime. <br/>
-**Order_Demand** to numeric. Order_Demand values include negative numbers that are formated as "(number)". These values need to be reformatted as "-number" before being converted to numeric.

2/ Warehouse and category information do not contribute to forecasting and will be removed. <br/>
- **Warehouse:** As per my understanding from data description, the four central warehouses serve demands for the same region. Therefore, the forecast will be performed for total demands in the region, without consideration of warehouse information. Product quantity per warehouse will be calculated based on other information like warehouse capacity and transportation cost, which is not in the scope of this demand forecasting. <br/>
- **Product_Category:** There is no other information to understand about product characteristics in each category. Meanwhile, the number of products per category is high, which makes visualization on data patterns of products per category impossible. Thus, it is difficult to understand products' data patterns in accordance with category number. Category column does not well explain demands in this dataset, and thus can be dropped.

In [3]:
# Format Date to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Format Order_Demand to numeric
data['Order_Demand'] = data['Order_Demand'].str.replace('(','-')
data['Order_Demand'] = data['Order_Demand'].str.replace(')','')
data['Order_Demand'] = pd.to_numeric(data['Order_Demand'])

# Drop warehouse and category columns
data = data.drop(columns=['Warehouse','Product_Category'])

# Sort data by period
data = data.sort_values('Date').reset_index().drop('index',axis=1)

In [4]:
print("Data Types")
print(data.dtypes)
print("\t")

print(data.head())
print("\t")

print("Data Dimension",data.shape)
print("\t")

print("The number of products is",len(data['Product_Code'].value_counts().index))
print("Period range is from",data['Date'].min(),"to", data['Date'].max())
print("Order Qty is from",data['Order_Demand'].min(),"to", data['Order_Demand'].max())
print("\t")

print('Number of missing values by column',[sum(data[i].isnull()) for i in data.columns])
print('All missing values are in Date column.')

Data Types
Product_Code            object
Date            datetime64[ns]
Order_Demand             int64
dtype: object
	
   Product_Code       Date  Order_Demand
0  Product_0965 2011-01-08             2
1  Product_1724 2011-05-31           108
2  Product_1521 2011-06-24         85000
3  Product_1521 2011-06-24          7000
4  Product_1507 2011-09-02          1250
	
Data Dimension (1048575, 3)
	
The number of products is 2160
Period range is from 2011-01-08 00:00:00 to 2017-01-09 00:00:00
Order Qty is from -999000 to 4000000
	
Number of missing values by column [0, 11239, 0]
All missing values are in Date column.


##### Negative Values
Order demands in this dataset include negative values. These can be either order adjustments or order returns. <br/>
Let's take a closer look into this by examining some products that have negative demand values.

In [5]:
# Check percentage of negative values vs postive values
# Extract negative values and aggregate by product
negative = data.loc[data['Order_Demand'] < 0]
negative_pv = pd.pivot_table(negative, values='Order_Demand',index=['Product_Code'], aggfunc=np.sum
                            ).rename(columns={'Order_Demand':'Total_Neg'})

# Extract positve values and aggregate by product
positive = data.loc[data['Order_Demand'] > 0]
positive_pv = pd.pivot_table(positive, values='Order_Demand',index=['Product_Code'], aggfunc=np.sum)

# Add a column with corresponding total positive value by product and calculate percentage, sort = desc
negative_pv['Total_Pos'] = positive_pv.loc[positive_pv.index.isin(negative_pv.index),]
negative_pv['Percentage'] = abs(negative_pv['Total_Neg'])*100/negative_pv['Total_Pos']
negative_pv = negative_pv.sort_values('Percentage', ascending = False)

print("Percentage ranges from",negative_pv['Percentage'].min(),"to",negative_pv['Percentage'].max())

Percentage ranges from 0.0011126440874093195 to 82.183908045977


In [6]:
# Examine demand of several products
print("Demand of Product_0319")
print(data.loc[data['Product_Code'] == 'Product_0319'].head(10))
print("\t")
print("Demand of Product_0568")
print(data.loc[data['Product_Code'] == 'Product_0568'].head(10))

Demand of Product_0319
        Product_Code       Date  Order_Demand
159537  Product_0319 2012-10-10            12
185920  Product_0319 2012-11-26           -10
316157  Product_0319 2013-07-15           300
383771  Product_0319 2013-10-30             3
383791  Product_0319 2013-10-30             5
385642  Product_0319 2013-10-31             5
486803  Product_0319 2014-04-14             2
524745  Product_0319 2014-06-19          -276
802112  Product_0319 2015-10-08             1
831542  Product_0319 2015-11-30             1
	
Demand of Product_0568
        Product_Code       Date  Order_Demand
691399  Product_0568 2015-03-26             4
700218  Product_0568 2015-04-09            20
759382  Product_0568 2015-07-23             1
759395  Product_0568 2015-07-23            30
771202  Product_0568 2015-08-12             1
771205  Product_0568 2015-08-12            95
817602  Product_0568 2015-11-05            10
817713  Product_0568 2015-11-05           160
822702  Product_0568 2015-11-13 

From the examples, it can be seen that negative values indicate both order adjustment/cancellation, and returns. Reasons: <br/>
- *Product_0319*: negative demands came separately in months after order shipment and dates refer to shipping dates (and dates when products are shipped back to warehouses in case of returns), not order dates. They are highly likely to be returns. <br/>
- *Product_0568*: negative demands are more likely to be order cancellation, because there is another observation with the same quantiy (in an opposite sign) and on the same date. 

Returns can stem from different reasons, which includes returns due to wrong shipment, returns due to bad quality. Concerning returns due to bad quality, products can be either fixed for future shipment or be destroyed. <br/>
Returns should be forecasted separately and returns forecasting results can be processed together with demand forecasting based on company's policies/practices on returns.

Due to information limitation on returns, I make another assumption: <br/>
**A3: Returned products are due to quality issues and are destroyed.** <br/>
With this assumption, return quantities are removed from the dataset and no forecasting is necessarily made on return quantity.

Negative values that indicate order cancellations will be left in the dataset and offset with other values. However, in this dataset, it's difficult to differentiate between order adjustment, order cancellation, and returns. Thus, I make one more assumption: <br/>
**A4: When customers want to adjust order quantity, system will record an order with all same information (date, order number) as the old order, but with order quantity of opposite sign (negative value), and then create another order (with the same order number) with updated order quantity and updated date (if any)**. <br/>
With this assumption, an observation with negative order quantity is order cancellation if there is another order for that product with the same date and same order quantity with positive value. Otherwise, the negative values are returns.

The dataset is then processed to remove all returns observations.

In [7]:
### Steps to remove returns:
# This concerns Date values, thus observations with missing valuse in Date column need to be removed first.
# Observations with missing values are extracted into a new table. Investigation into missing values will be 
# performed later.
data_null = data.loc[data['Date'].isnull() == True,]
data = data.dropna(how='any',axis=0)
[sum(data[i].isnull()) for i in data.columns]

# Create a column with absolute values of order_demand column
data['Abs_Demand'] = abs(data['Order_Demand'])

# Remove all rows that have the same products, dates, and absolute values of demand
data_nodup = data.drop_duplicates(subset = ['Product_Code','Date','Abs_Demand'], keep = False)

In [8]:
# Extract all removed duplicates rows to a new dataframe and aggregate by product and date 
# By aggregation, cancelled orders and the corresponding orders will be offset and totalled 0. 
# In cases where the sums are lower than 0, these observations are actually returns and will be removed.
# (For example, two returns are on the same day -> same absolute values -> extracted this dataset of duplicates.)
duplicates = data.iloc[~data.index.isin(data_nodup.index)]
duplicates = pd.pivot_table(duplicates, values='Order_Demand',index=['Product_Code','Date'],aggfunc=np.sum
                           ).reset_index()
duplicates = duplicates.loc[duplicates['Order_Demand']>0]

In [9]:
# Remove all rows of duplicates in the original dataset, and drop Abs_Demand column
# Remove all negative values
data = data.drop_duplicates(subset = ['Product_Code','Date','Abs_Demand'], keep = False)
data = data.drop('Abs_Demand',axis=1)
data = data.loc[data['Order_Demand']>0]

# Add duplicates table back to data
data = pd.concat([data,duplicates], ignore_index = True)

# Check to see if any negative values remain
data.loc[data['Order_Demand'] <0] # no rows

,Product_Code,Date,Order_Demand


##### Missing Values

In [10]:
# The number of products have null values
print("The number of products with null is",len(data_null['Product_Code'].value_counts().index))

The number of products with null is 82


With all the assumptions above, it is impossible to further analysis observations that have missing dates. 
There is no other option but to omit those observations.
The current dataset at this step already have all null values omitted.

##### Aggregate data by month
Forecasting is performed at monthly horizons, thus the dataset should first be aggregated by month. Date is extracted with Month & Year only.

In [11]:
data['Date'] = data['Date'].dt.to_period('M')
data = data.rename(columns = {"Date": 'Period'})
data = data.groupby(['Product_Code','Period'])['Order_Demand'].sum().reset_index().sort_values('Period'
            ).reset_index().drop('index',axis=1)
data.head()

,Product_Code,Period,Order_Demand
0,Product_0965,2011-01,2
1,Product_1724,2011-05,108
2,Product_1521,2011-06,92000
3,Product_1507,2011-09,1250
4,Product_0608,2011-09,5


In [12]:
# Check to see if periods in dataset is continuous
# Create a duration with continuous periods
full_period = pd.date_range('2011-01-01','2016-12-31', freq='MS').to_period('M')
full_period = set(full_period)
data_period = set(data['Period'])
full_period.difference(data_period)
# The missing periods are 5 months in 2011, including Feb, Mar, Apr, Jul, and Aug.
# There are various possible reasons for the missing periods: No demands are in these months, warehouses to be
# closed in these months for some reason, missing data in these periods, etc.
# To ensure that the training data will not be misleading, all data before Sep 2011 will be removed.
data = data.loc[data['Period'] > '2011-08']

##### Check to see which products are eligible for forecasting

Some analysis is done to see if any products should be excluded from the forecasting. <br/>
Several criteria are as below: <br/>
**1/** All data in Jan, 2017 should be removed; otherwise, model interpretation will be misled. <br/>
Reason: The latest date in this dataset is 2017/01/09 while forecasting is for monthly horizon. <br/>
**2/** Products that have no order quantities in 2016 will be removed. <br/>
Reason: It is highly likely that the products have been stopped already and will have no future demand. The duration of 1 year helps cover the cases of seasonal products.  <br/>
**3/** Products with demand history less than 24 months will be removed. <br/>
Reason: A minimum of 2 years' data is required to forecast trends and seasonality using statistical forecasting methods. Also, in cases when history is too short (6 months for example), the products are likely to be new products and forecasting methods for new products are different. 

In [13]:
# Criteria 1: Remove data in Jan, 2017
data = data.loc[data['Period']<'2017-01']

In [14]:
# Criteria 2: Remove stopped products
latest_datamonth = data.groupby('Product_Code')['Period'].max().reset_index()
latest_datamonth = latest_datamonth.loc[latest_datamonth['Period'] > '2015-12']
data = data.loc[data['Product_Code'].isin(latest_datamonth['Product_Code'])]

In [15]:
# Criteria 3: Remove new products
duration_data = data.groupby('Product_Code').agg({'Period': ['min', 'max']}).reset_index()
duration_data['Duration'] = (duration_data[('Period', 'max')] - duration_data[('Period', 'min')]
                            ).apply(attrgetter('n')) + 1
duration_data = duration_data.loc[duration_data['Duration'] > 24 ]
data = data.loc[data['Product_Code'].isin(duration_data['Product_Code'])]

##### Construct time series in a columnar format

In [16]:
data = pd.pivot_table(data, values = 'Order_Demand', index = 'Period', columns = 'Product_Code',aggfunc=np.sum
                     ).reset_index().rename_axis("", axis="columns")

#Fill in missing values with 0. Months with missing values are implied to have zero demands.
data = data.fillna(0)
data = data.set_index('Period')
data.head()

,Product_0001,Product_0002,Product_0003,Product_0004,Product_0005,Product_0006,Product_0007,Product_0008,Product_0009,Product_0010,...,Product_2163,Product_2164,Product_2165,Product_2166,Product_2167,Product_2168,Product_2169,Product_2170,Product_2171,Product_2172
Period,,,,,,,,,,,,,,,,,,,,,
2011-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-12,300.0,221000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,107.0,3.0,20.0,4.0,2.0,2.0,0.0,0.0
2012-01,9700.0,65000.0,400.0,300.0,0.0,0.0,2100.0,4500.0,200.0,600.0,...,101.0,4.0,115.0,137.0,3844.0,463.0,509.0,18.0,31.0,100.0


In [17]:
# The dataset includes data for 2061 products, which takes Python too long to return model results.
# Thus, I sampled only several products with different patterns for demonstration purpose
data = data[['Product_0002','Product_0138','Product_0597','Product_0875','Product_2066',
               'Product_2091','Product_2127','Product_2165']]
data.shape

(64, 8)

##### Try with models

In [18]:
# function to split train and test set
# history length of a product starts from the month of first order, not all products have the same history length
# the function is to split train-test based on product's history length instead of dataset length
def train_test(data):
    myList = data.tolist()
    i = myList.index(next(filter(lambda x: x!=0, myList)))
    data = data.iloc[i:,]
    train = data[:int(0.7*(len(data)))]
    test = data[int(0.7*len(data)):]
    return train, test

# create data for forecasting
start = data.index.tolist()[-1] + 3
fcastperiods = 6  # forecast periods is subject to change by forecast users
full_period = [start + x for x in range(0,fcastperiods)]

##### ARIMA Model

In [24]:
%%capture 

# build function to run model for all columns
Arima = ['Arima']
ArimaFcastPerf = pd.DataFrame({'Models': Arima})

ArimaData = pd.DataFrame({'Period': full_period, 'Model': 'Arima'})

def arimafcast(data):
    for i in data.columns:
        try:
            train, test = train_test(data[i])
            model = auto_arima(train, start_p=0, start_q=0, m=1, trace=True, 
                               error_action='ignore', suppress_warnings=True)
            model.fit(train)
            pred = np.round(model.predict(n_periods=len(test)+3+fcastperiods-1))
            ArimaFcastPerf[i] = sqrt(mean_squared_error(test,pred[:-fcastperiods-2]))
            ArimaData[i] = pred[-fcastperiods:]
        except:
            ArimaFcastPerf[i] = np.nan
            ArimaData[i] = np.nan
    return ArimaFcastPerf, ArimaData
arimafcast(data)

##### Simple Exponential Smoothing

In [25]:
%%capture

# build function to run model for all columns
SES = ['SES']
SESFcastPerf = pd.DataFrame({'Models': SES})

SESData = pd.DataFrame({'Period': full_period, 'Model': 'SES'})

def sesfcast(data):
    for i in data.columns:
        try:
            train, test = train_test(data[i])
            model = SimpleExpSmoothing(train).fit()
            pred = np.round(model.forecast(len(test)+3+fcastperiods-1))
            SESFcastPerf[i] = sqrt(mean_squared_error(test,pred[:-fcastperiods-2]))
            SESData[i] = pred[-fcastperiods:].tolist()
        except:
            SESFcastPerf[i] = np.nan
            SESData[i] = np.nan
    return SESFcastPerf, SESData
sesfcast(data)

##### Double Exponential Smoothing
Note: Double & Triple Exponential Smoothing only apply to positive data values

In [26]:
%%capture

# build function to run model for all columns
DES = ['DES']
DESFcastPerf = pd.DataFrame({'Models': DES})

DESData = pd.DataFrame({'Period': full_period, 'Model': 'DES'})

def desfcast(data):
    for i in data.columns:
        try:
            train, test = train_test(data[i])
            if train.min() != 0: #Double & Triple Exponential Smoothing only applies to positive data values
                model = ExponentialSmoothing(train, trend='add', seasonal=None, damped=True
                                            ).fit(optimized=True,use_boxcox=True)
                pred = np.round(model.forecast(len(test)+3+fcastperiods-1))
                DESFcastPerf[i] = sqrt(mean_squared_error(test,pred[:-fcastperiods-2]))
                DESData[i] = pred[-fcastperiods:].tolist()
            else:
                DESFcastPerf[i] = np.nan
                DESData[i] = np.nan  
        except:
            DESFcastPerf[i] = np.nan
            DESData[i] = np.nan
    return DESFcastPerf, DESData
desfcast(data)

##### Triple Exponential Smoothing

In [27]:
%%capture

# build function to run model for all columns
TES = ['TES']
TESFcastPerf = pd.DataFrame({'Models': TES})

TESData = pd.DataFrame({'Period': full_period, 'Model': 'TES'})

def tesfcast(data):
    for i in data.columns:
        try:
            train, test = train_test(data[i])
            if train.min() != 0: 
                model = ExponentialSmoothing(train, seasonal_periods=12, 
                                             trend='add', seasonal='mul', damped=True
                                            ).fit(optimized=True,use_boxcox=True)
                pred = np.round(model.forecast(len(test)+3+fcastperiods-1))
                TESFcastPerf[i] = sqrt(mean_squared_error(test,pred[:-fcastperiods-2]))
                TESData[i] = pred[-fcastperiods:].tolist()
            else:
                TESFcastPerf[i] = np.nan
                TESData[i] = np.nan       
        except:
            TESFcastPerf[i] = np.nan
            TESData[i] = np.nan   
    return TESFcastPerf, TESData
tesfcast(data)

##### Compare and select the best model for each product

In [28]:
#Combine all performance tables
FcastPerf = pd.concat([ArimaFcastPerf,SESFcastPerf,DESFcastPerf,TESFcastPerf]).set_index('Models')
FcastPerf

,Product_0002,Product_0138,Product_0597,Product_0875,Product_2066,Product_2091,Product_2127,Product_2165
Models,,,,,,,,
Arima,118217.407066,1920.982657,4.339739,408.088226,239.215384,102.328280,165.067495,83.539779
SES,158304.234195,2229.192785,2.886751,544.968806,231.796737,102.033554,165.067495,83.539779
DES,117386.637451,1415.496733,NaN,NaN,NaN,NaN,NaN,72.961417
TES,192662.202227,1306.719376,NaN,NaN,NaN,NaN,NaN,108.229532


In [29]:
# Find best model for each product
Model = pd.DataFrame()

for i in FcastPerf.columns:
    # Find the one with lowest RMSE. Choose the first one in case of more than 1 min values.
    Model[i] = list([FcastPerf.loc[FcastPerf[i] == FcastPerf[i].min()].index[0]])
Model

,Product_0002,Product_0138,Product_0597,Product_0875,Product_2066,Product_2091,Product_2127,Product_2165
0,DES,TES,SES,Arima,SES,SES,Arima,DES


In [30]:
# Filter list of products per model
model_dict = {}
model_list = ['Arima', 'SES', 'DES', 'TES']
for i in model_list:
    model_dict[i] = []

for i in Model.columns:
    for j in model_list:
        if any(Model[i] == j):
            model_dict[j].append(i)
model_dict

{'Arima': ['Product_0875', 'Product_2127'],
 'SES': ['Product_0597', 'Product_2066', 'Product_2091'],
 'DES': ['Product_0002', 'Product_2165'],
 'TES': ['Product_0138']}

In [31]:
# Extract forecast from equivalent models to the final forecast dataframe
AllForecast = pd.concat([ArimaData, SESData, DESData, TESData])
FinalForecast = pd.DataFrame({'Period': full_period})

for i in model_list:
    for j in model_dict[i]:
        FinalForecast[j] = AllForecast.loc[AllForecast['Model'] == i][j]
FinalForecast = FinalForecast[sorted(FinalForecast)]

In [32]:
# Save final forecast
#FinalForecast.to_csv("FinalForecast.csv")
FinalForecast

,Period,Product_0002,Product_0138,Product_0597,Product_0875,Product_2066,Product_2091,Product_2127,Product_2165
0,2017-03,138027.0,3861.0,3.0,204.0,19.0,94.0,182.0,98.0
1,2017-04,138027.0,2063.0,3.0,204.0,19.0,94.0,182.0,98.0
2,2017-05,138027.0,1894.0,3.0,204.0,19.0,94.0,182.0,98.0
3,2017-06,138027.0,2547.0,3.0,204.0,19.0,94.0,182.0,98.0
4,2017-07,138027.0,1725.0,3.0,204.0,19.0,94.0,182.0,98.0
5,2017-08,138027.0,3181.0,3.0,204.0,19.0,94.0,182.0,98.0


**Limitations of this forecast:** <br/>
1/ *Scattered data*: The data scattered too much, and not really appropriate for statistical forecasting methods. That might be the result that the forecasting results do not look really good. <br/>
2/ *No external factors explained:* The forecast is based on historical data only and doesnt incorporate any external information like economic conditions, product price policy changes, future promotions, etc. <br/>
3/ *No consideration of returns impact:* Order returns can make negative impact on future demand. For example, customers may leave, which leads to a drop in future demand. However, there is not information to make any assumptions about this impact in this forecasting. <br/>
4/ *Omitted missing values:* It's impossible to measure the impact of removing missing values in this dataset. <br/> 